In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)
ALL_IN_ONE_WORKER = True

In [2]:
@dataclass
class Message:
    content: str

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address = "localhost:50051")
host.start()

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper = Tool(name = "internet_search", func = serper.run, description="Useful when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the cons of AutoGen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)

In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")

In [8]:
response = await worker.send_message(Message(content="Go"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some compelling reasons to choose AutoGen for your AI Agent project:

1. **Multi-Agent Collaboration**: AutoGen is specifically designed for scenarios where multiple AI agents work together. This makes it ideal for complex tasks that require collaborative decision-making and interaction.

2. **Simplified Development**: It allows engineers to create task-oriented agents without starting from scratch, reducing the time and effort required to develop AI applications.

3. **Customizable Agents**: With AutoGen, you can define multiple agents with specific roles and capabilities, tailoring the system to suit the needs of your project.

4. **Effective Communication**: The framework uses natural language for inter-agent communication, which simplifies coordination and reduces complexity compared to traditional methods.

5. **Robust Backing**: Developed by Microsoft Research, AutoGen benefits from strong support and frequent usage in various applications, providing confidence in its reliability and performance.

6. **Open Source**: Being an open-source framework, it allows developers to access and modify the code, fostering community collaboration and innovation.

7. **No Revenue Pressure**: As a project funded by Microsoft, AutoGen does not face the same revenue pressures as commercial solutions, allowing it to focus on providing innovative features.

8. **Enterprise Ready**: Its capabilities are well-suited for enterprise AI development, enabling the orchestration, optimization, and automation of complex workflows.

These advantages make AutoGen a promising choice for building effective AI Agent systems. 

TERMINATE

## Cons of AutoGen:
### Reasons in Favor of Choosing AutoGen:
1. **Multi-Agent Collaboration**: AutoGen excels in scenarios requiring dynamic collaboration among multiple AI agents, making it ideal for complex tasks involving decision-making and research.
2. **Robust Tool Integration**: It offers strong capabilities for integrating with various tools, APIs, and custom functions, allowing agents to perform complex tasks autonomously and interact seamlessly with external resources.
3. **Flexibility and Customization**: AutoGen allows for high customization of agents and automation processes, providing fine-grained control over functionalities.
4. **Compatibility with Different LLMs**: The framework supports various large language models (LLMs), enabling the development of versatile applications.
5. **Strong Backing by Microsoft**: As a Microsoft framework, AutoGen benefits from substantial resources, leading to ongoing improvements and a broad usage base.

### Cons of AutoGen:
1. **Limited Custom Integrations**: AutoGen can be less flexible when it comes to highly complex or unstructured interactions between agents, which may restrict its usage in some innovative applications.
2. **Steeper Learning Curve**: Its open-source nature and focus on developer-centric customization mean it can be more challenging to learn, particularly for non-technical users. Lack of a visual builder or no-code options can make initial development more difficult.
3. **Potential for Conversation Issues**: User experiences indicate that multi-agent interactions can sometimes derail, requiring careful management of agent communication and protocols.
4. **Less Suitable for Simple Workflows**: If a project involves straightforward, well-defined tasks, other frameworks might offer better performance due to AutoGen's complexity.

TERMINATE



## Decision:

Based on the pros and cons presented, I recommend using AutoGen for the project. The strengths of AutoGen, such as its capabilities for multi-agent collaboration, simplified development process, customization options, effective communication, and strong backing from Microsoft Research, align well with the needs of complex AI Agent applications. 

While there are some drawbacks, such as the steeper learning curve and potential conversation issues, these can be managed with a capable development team. The advantages greatly outweigh the cons, particularly for projects involving intricate workflows and collaborative tasks, making AutoGen a promising and suitable choice.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()